In [16]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm


In [17]:

# Directories for training and testing data
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/validation'


In [18]:

# Function to create a dataframe for images and their labels
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir, label)):
            image_paths.append(os.path.join(dir, label, imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths, labels

In [19]:

# Create dataframe for training and testing data
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed
angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [20]:

# Function to extract features from images
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, color_mode='grayscale')
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48, 48, 1)
    return features


In [21]:

# Extract features for training and testing datasets
train_features = extract_features(train['image'])
test_features = extract_features(test['image'])

# Normalize the data by dividing by 255
x_train = train_features / 255.0
x_test = test_features / 255.0

100%|██████████| 7066/7066 [00:38<00:00, 181.64it/s]


In [22]:

# Label encoding and one-hot encoding the labels
le = LabelEncoder()
le.fit(train['label'])

y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)


In [23]:

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=30,        # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,    # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,   # randomly shift images vertically (fraction of total height)
    zoom_range=0.2,           # randomly zoom in on images
    horizontal_flip=True,     # randomly flip images horizontally
    fill_mode='nearest'       # filling in new pixels when images are rotated or shifted
)

# Use the data generator on the training set
train_datagen = datagen.flow(x_train, y_train, batch_size=128)

# No data augmentation for the validation/test set
test_datagen = ImageDataGenerator()
test_datagen_flow = test_datagen.flow(x_test, y_test)


In [24]:

# Define the model architecture
model = Sequential()
# Convolutional layers
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Flatten())
# Fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# Output layer
model.add(Dense(7, activation='softmax'))


w:\Important Documents\Manan\2 UK\MSc\Dissertation\archive\env\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [25]:

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [26]:

# Train the model with augmented data
model.fit(train_datagen, epochs=100, validation_data=test_datagen_flow)


Epoch 1/100


w:\Important Documents\Manan\2 UK\MSc\Dissertation\archive\env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


226/226 ━━━━━━━━━━━━━━━━━━━━ 91s 392ms/step - accuracy: 0.2353 - loss: 1.8388 - val_accuracy: 0.2583 - val_loss: 1.8095
Epoch 2/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 87s 385ms/step - accuracy: 0.2502 - loss: 1.8078 - val_accuracy: 0.2540 - val_loss: 1.7878
Epoch 3/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 87s 386ms/step - accuracy: 0.2473 - loss: 1.7922 - val_accuracy: 0.2577 - val_loss: 1.7787
Epoch 4/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 86s 382ms/step - accuracy: 0.2504 - loss: 1.7854 - val_accuracy: 0.2924 - val_loss: 1.7308
Epoch 5/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 86s 380ms/step - accuracy: 0.2666 - loss: 1.7611 - val_accuracy: 0.3020 - val_loss: 1.7028
Epoch 6/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 86s 379ms/step - accuracy: 0.2791 - loss: 1.7465 - val_accuracy: 0.3258 - val_loss: 1.6653
Epoch 7/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 81s 358ms/step - accuracy: 0.2863 - loss: 1.7308 - val_accuracy: 0.3372 - val_loss: 1.6548
Epoch 8/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 82s 364ms/step - accuracy: 0.2891 - loss: 1.727

In [27]:

# Save the model
model_json = model.to_json()
with open("emotiondetector4.json", 'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector4.h5")


In [28]:

# Load the saved model and weights
json_file = open("emotiondetector4.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("emotiondetector4.h5")


In [29]:

# Prediction function
label = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

def ef(image):
    img = load_img(image, color_mode='grayscale')
    feature = np.array(img)
    feature = feature.reshape(1, 48, 48, 1)
    return feature / 255.0

In [30]:

# Example prediction
image = 'images/train/sad/42.jpg'
print("original image is of sad")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is", pred_label)


original image is of sad
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
model prediction is sad
